In [24]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
import os
import requests
from requests.utils import quote
import json
import re

API_KEY = "74cCBsjRIn+Q3tBRgGFBng==6qu7n1HlTjvzVi2r"
NURT_API_ID = "73bd8bbc"
NURT_API_KEY = "969aa90fceed449d4137b10737588939"
CAL_NINJA = "2oYO3lQwGjIr9RgV+oRa6w==pB0UmSVSaooa8BXB"

MACROS = ['calories', 'total_fat', 'saturated_fat', 'cholesterol', 'sodium', 'total_carbohydrate', 'dietary_fiber', 'sugars', 'protein', 'potassium']
MACROS_CALN = ['calories', 'fat_total_g', 'fat_saturated_g', 'protein_g', 'sodium_mg', 'potassium_mg', 'cholesterol_mg', 'carbohydrates_total_g', 'fiber_g', 'sugar_g']
FOOD_LIST = dict()  
API_CALLS = 3118  
MAX_CALLS = 9500

In [25]:
# Get Elasticsearch host from environment variables
elasticsearch_host = "http://localhost:9200"
elasticsearch_username = "elastic"
elasticsearch_password = "p5FE3c=alPhGd20o14bx"

es = Elasticsearch(elasticsearch_host, basic_auth=(elasticsearch_username, elasticsearch_password))
# Connect to Elasticsearch
try:
    if es.ping():
        print("Successfully connected to Elasticsearch!")
    else:
        print("Failed to connect to Elasticsearch.")
except Exception as e:
    print(f"Error connecting to Elasticsearch: {e}")

Successfully connected to Elasticsearch!


In [77]:
def fetch_recipes(query, offset = 0):
    """Fetch recipes from API-Ninjas and return JSON response."""
    api_url = f"https://api.api-ninjas.com/v1/recipe?query={query}&offset={offset}"
    headers = {"X-Api-Key": API_KEY}
    
    response = requests.get(api_url, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return []

def store_recipes_in_elasticsearch(recipes, idx = 'recipes'):
    """Store the fetched recipes in Elasticsearch."""
    for i, recipe in enumerate(recipes):
        es.index(index=idx, id=i+1, document=recipe)
    print("Recipes successfully stored in Elasticsearch!")

def search_recipes(keyword):
    """Search for recipes in Elasticsearch."""
    results = es.search(index="recipe_box", query={
        "match": {
            "title": keyword  # Searches in the recipe title field
        }
    })
    return results["hits"]["hits"]

In [ ]:
# Example: Fetch & Store "Italian Wedding Soup"
query = "italian wedding soup"
recipes = fetch_recipes(query)
print(recipes)

if recipes:
    store_recipes_in_elasticsearch(recipes)

In [78]:
# Example search
search_results = search_recipes("avocado")
print(search_results)

for recipe in search_results:
    print(recipe["_source"])

[{'_index': 'recipe_box', '_id': '142', '_score': 3.1802986, '_source': {'title': 'Chicken-Avocado Salad', 'labels': ['Avocado'], 'rating': 0, 'ingredients': [{'name': 'Finely chopped cold cooked chicken', 'qty': 2.0, 'unit': 'c'}, {'name': 'Finely chopped celery', 'qty': 0.5, 'unit': 'c'}, {'name': 'Fresh lemon juice', 'qty': 1.0, 'unit': 'tb'}, {'name': 'Hot pepper sauce', 'qty': 1.0, 'unit': 'ds'}, {'name': 'Mayonnaise', 'qty': 0.5, 'unit': 'c'}, {'name': 'Ripe avocados', 'qty': 2.0, 'unit': 'lg'}, {'name': 'cooked eggs; sliced', 'qty': 2.0, 'unit': 'Hard'}], 'macros': {'calories': 1701.3, 'fat_total_g': 147.8, 'fat_saturated_g': 27.000000000000004, 'protein_g': 60.3, 'sodium_mg': 1125, 'potassium_mg': 690, 'cholesterol_mg': 576, 'carbohydrates_total_g': 29.199999999999996, 'fiber_g': 19.700000000000003, 'sugar_g': 4.4}, 'instructions': ['combine all ingredients except avocados and eggs', 'Have avocadoes and remove seeds', 'Scoop out pulp and reserve avacado shell', 'Dice pulp and a

In [51]:
def fetch_nutrition(query):
    """Fetch recipes from Nutritionix and return JSON response."""
    # URL encode the query to handle special characters
    encoded_query = quote(query)
    api_url = f"https://trackapi.nutritionix.com/v2/search/instant/?query={encoded_query}"
    headers = {
        'Content-Type': 'application/json',
        'X-Api-Key': CAL_NINJA,
               
    }
    
    response = requests.get(api_url, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return []
    
def fetch_nutrition_NL(query):  
    url = 'https://api.calorieninjas.com/v1/nutrition?query='
    headers = {
        'Content-Type': 'application/json',
        'X-Api-Key': CAL_NINJA,
        
    }

    # Set up the body of the request
    body = {
        "query": query  
    }

    response = requests.get(url + query, headers=headers)

    # Check for errors
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")

def process_NL_info(ing, info, total_macro):
    if info is not None:
        info = info['foods'][0]
        ing_serv_info = dict()

        if any(char.isdigit() for char in ing):
            ing_name = info['food_name']
            ing_serv_info["qty"] = info['serving_qty']
            ing_serv_info["unit"] = info['serving_unit']

            for macro in MACROS:
                if info['nf_' + macro] is not None:
                    if macro in total_macro.keys():
                        total_macro[macro] += info['nf_' + macro]
                    else:
                        total_macro[macro] = info['nf_' + macro]

            if info['food_name'] not in FOOD_LIST.keys():
                unit_macros = dict()
                for macro in MACROS:
                    if info['nf_' + macro] is not None and ing_serv_info["qty"] is not None and ing_serv_info["qty"] > 0:
                        unit_macros[macro] = info['nf_' + macro] / ing_serv_info["qty"]
                    else:
                        print(f'Could not parse {ing_name} {macro}')
                        unit_macros[macro] = 0
                FOOD_LIST[(info['food_name'], ing_serv_info["unit"])] = unit_macros
        else:
            ing_name = ing
            ing_serv_info["qty"] = 0
            ing_serv_info["unit"] = "N/A"

        return ing_name, ing_serv_info
    else:
        print(f'Error in finding {ing}')

def fetch_calN_nutrition(query):
    api_url = 'https://api.calorieninjas.com/v1/nutrition?query='
    response = requests.get(api_url + query, headers={'X-Api-Key': CAL_NINJA})
    if response.status_code == requests.codes.ok:
        return response.json()['items']
    else:
        print("Error:", response.status_code, response.text)

def split_string(input_string):
    # Regular expression pattern to match amount, unit, and name#
    pattern = r'(?P<amount>[\d\s/.]+)(?:\s+(?P<unit>[a-zA-Z]+))?\s+(?P<name>.+)'
    
    match = re.match(pattern, input_string)
    if match:
        return match.group('amount'), match.group('unit'), match.group('name')
    else:
        return None, None, None  # Return None if the pattern doesn't match
    
def convert_to_float(fraction_str):
    # Split the string into whole number and fraction
    parts = fraction_str.split()
    
    # Initialize the total
    total = 0.0
    
    # Process the whole number part if it exists
    if len(parts) > 0:
        if '/' not in parts[0]:
            total += float(parts[0])
        else:
            numerator, denominator = map(float, parts[0].split('/'))
            total += numerator / denominator 
    
    # Process the fractional part if it exists
    if len(parts) > 1:
        # Split the fraction into numerator and denominator
        pts = parts[1].split('/')
        if len(pts) > 1:
            numerator, denominator = map(float, pts)
            total += numerator / denominator  # Add the fraction to the total
    
    return total
    
def process_calN_info(ing, total_macro):
    global API_CALLS
    info = fetch_calN_nutrition(ing)
    API_CALLS += 1

    if API_CALLS > MAX_CALLS:
        return
    
    if info is not None:
        amt, unit, name = split_string(ing)
        ing_serv_info = dict()
        
        if amt is not None:
            ing_name = name
            ing_serv_info["name"]  = name
            ing_serv_info["qty"] = convert_to_float(amt)
            ing_serv_info["unit"] = unit

            if len(info) != 0:
                info = info[0]
                for macro in MACROS_CALN:
                    if macro in info.keys():
                        if macro in total_macro.keys():
                            total_macro[macro] += info[macro]
                        else:
                            total_macro[macro] = info[macro]

                if ing_name not in FOOD_LIST.keys() and MACROS_CALN[0] in info.keys():
                    unit_macros = dict()
                    for macro in MACROS_CALN:
                        if macro in info.keys() and ing_serv_info["qty"] is not None and ing_serv_info["qty"] > 0:
                            unit_macros[macro] = info[macro] / ing_serv_info["qty"]
                        else:
                            unit_macros[macro] = 0
                    FOOD_LIST[(ing_name, ing_serv_info["unit"])] = unit_macros
        
            return ing_name, ing_serv_info
        else:
            return None, None
    else:
        print(f'Error Processing {ing}')
        return None, None

In [52]:
def process_recipe(recipe):
    global API_CALLS
    recipe_json = dict()
    # get recipe name
    recipe_json['title'] = recipe['title']

    # generate labels
    recipe_json['labels'] = []

    # initialize rating
    recipe_json['rating'] = 0

    # get ingredient info
    ingreds = recipe['ingredients'].split('|')
    ingredients = []

    total_macro = dict()

    for ing in ingreds:
        #info = fetch_nutrition_NL(ing)
        #name, item_info = process_NL_info(ing, info, total_macro)
        
        name, item_info = process_calN_info(ing, total_macro)
        if name is not None:
            ingredients.append(item_info) 
        

    recipe_json['ingredients'] = ingredients
    recipe_json['macros'] = total_macro.copy()

    # split instructions
    recipe_json['instructions'] = list(filter(lambda item: len(item) >= 5, recipe['instructions'].split('. ')))
    

    return recipe_json

In [ ]:
recorded_recipes = []
formatted_json = []

In [72]:
recipe_words = [
    # Main Ingredients
    "Chicken", "Beef", "Pork", "Fish", "Shrimp", "Tofu", "Egg", "Cheese", 
    "Tomato", "Onion", "Garlic", "Mushroom", "Spinach", "Broccoli", "Avocado"
    
    ]
"""
    # Cuisine Types
    "Italian", "Mexican", "Chinese", "Indian", "Thai", "Greek", "French", 
    "Japanese", "Korean", "Mediterranean"

    # Dish Types
    "Soup", "Salad", "Stew", "Stir-fry", "Casserole", "Pasta", "Sandwich", 
    "Wrap", "Curry", "Bowl", "Pizza", "Tacos", "Burrito", "Pie", "Cake",

    # Cooking Methods
    "Baked", "Roasted", "Grilled", "Fried", "Steamed", "Boiled", "Sautéed", 
    "Slow-cooked", "Smoked", "Glazed",

    # Descriptive Words
    "Spicy", "Sweet", "Creamy", "Crispy", "Savory", "Juicy", "Tender", 
    "Crunchy", "Zesty", "Hearty",

    # Dietary Labels
    "Vegan", "Vegetarian", "Gluten-Free", "Low-Carb", "Keto", "Paleo", 
    "Dairy-Free", "High-Protein"""


count = 0
for word in recipe_words:
    recipes = fetch_recipes(word.lower())

    for recipe in recipes:
        rep = recipe['title'] + recipe['ingredients']
        if rep not in recorded_recipes:
            recipe_json = process_recipe(recipe)
            if API_CALLS > MAX_CALLS:
                break
            recipe_json['labels'].append(word)
            formatted_json.append(recipe_json)
            recorded_recipes.append(rep)
    
    print(f'Number of Recipes Recorded for {word}: 10. Adding to Elasticsearch...' )

    if API_CALLS > MAX_CALLS:
        break

SSLError: HTTPSConnectionPool(host='api.api-ninjas.com', port=443): Max retries exceeded with url: /v1/recipe?query=italian&offset=0 (Caused by SSLError(SSLCertVerificationError(1, "[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: Hostname mismatch, certificate is not valid for 'api.api-ninjas.com'. (_ssl.c:1006)")))

In [1]:
print(API_CALLS)
print(len(formatted_json[start_idx:]))

NameError: name 'API_CALLS' is not defined

In [69]:
start_idx = 147
for recipe, i in zip(formatted_json[start_idx:], range(len(formatted_json[start_idx:]))):
    name = recipe["title"]
    es.index(index='recipe_box', id=start_idx + i, document=recipe)

print(start_idx + i)

Adding Emeril's Shrimp Pie
Adding Emerilized Barbecued Shrimp
Adding Endive Spears with Guacamole, Bay Shrimp, and Salsa
Adding Fri-Dinner: Hot-And-Sour Shrimp Soup
Adding Fried Eggplant with Creamed Shrimp and Crab
Adding Golden Tofu Wrap with Warm Asian Slaw
Adding Green Peppers And Tofu Sauteed with Miso
Adding Mediterranean Tofu Casserole
Adding Hot and Sour Garlic Chive Soup with Tofu
Adding Chicken, Tofu, and Vegetable Stir-Fry
Adding Cellophane Noodles with Tofu and Veggies
Adding Cajun Tofu And Roasted Red Pepper Pizza
Adding Chicken Soup with Tofu Cubes and Slivered Pork
Adding Chocolate Tofu Frosting -Lf
Adding Chocolate Tofu Pie
Adding Ella's Vegetable and Meat Egg Rolls
Adding Emeril's Beggar Purses Wrapped in a Truffle Crepe
Adding Emeril's Crab Meat Deviled Eggs
Adding English Muffin W/ham Egg
Adding Fresh Mushrooms with Eggplant and Tomato
Adding Fresh Veggie Basil Salad
Adding Fresh Veggie Pizza Bars
Adding Fried 100-Year Old Eggs
Adding Fried Egg
Adding Fried Egg Sauce

In [4]:
# Populate with data, index username for force search, and store password securely
users = [
    {
        "username": "user1",
        "password": "password1",
        "email": "user1@example.com",
        "first_name": "John",
        "last_name": "Doe"
    },
    {
        "username": "user2",
        "password": "password2",
        "email": "user2@example.com",
        "first_name": "Jane",
        "last_name": "Doe"
    }
]

for user in users:
    es.index(index='users', id=user["email"], document={
        "username": user["username"],
        "email": user["email"],
        "first_name": user["first_name"],
        "last_name": user["last_name"],
        "password": user["password"]  # Store password securely
    }, routing=user["email"])  

In [34]:
if es.indices.exists(index='users'):
    es.indices.delete(index='users')

# Create a new users index with the specified mapping
es.indices.create(index='users', body={
    "mappings": {
        "properties": {
            "username": {"type": "keyword"},
            "email": {"type": "keyword"},  # Use keyword type for email
            "first_name": {"type": "text"},
            "last_name": {"type": "text"},
            "password": {"type": "keyword"}  # Consider using a different type for hashed passwords
        }
    }
})

C:\Users\13307\AppData\Local\Temp\ipykernel_47840\403615161.py:5: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index='users', body={


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'users'})

In [36]:
username = "eat"
password = "eat!"
email = "eat@gmail.com"
firstName = "Eat"
lastName = "EATS"

user_data = {
    "username": username,
    "email": email,
    "first_name": firstName,
    "last_name": lastName,
    "password": password  # Store password securely
}

try:
    es.index(index='users', id=user_data['email'], document=user_data, routing=user_data['email'])  # Adding the user to the 'users' index

    new_pantry = {
        "user_id": email,
        "items": []
    }

    es.index(index='pantry', document=new_pantry)
except Exception as e:
    print("Error adding user to Elasticsearch:", e)

es.indices.refresh(index='users')  # Refresh the index after adding the user

# Search for the user in Elasticsearch
es_query = {
    "query": {
        "bool": {
            "must": [
                {"term": {"username": username}},  # Assuming username is indexed
                {"term": {"password": password}}  # Using term query for exact match
            ]
        }
    }
}

try:
    results = es.search(index="users", body=es_query)  # Assuming users are stored in 'users' index
    print(results)
    if results["hits"]["total"]["value"] > 0:
        user_email = results["hits"]['hits'][0]['_source']['password']
        print("User email:", user_email)
        # Delete the user after running the search
        es.delete(index='users', id=user_email)
except Exception as e:
    print("Error querying Elasticsearch:", e)



{'took': 4, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 0.5753642, 'hits': [{'_index': 'users', '_id': 'eat@gmail.com', '_score': 0.5753642, '_routing': 'eat@gmail.com', '_source': {'username': 'eat', 'email': 'eat@gmail.com', 'first_name': 'Eat', 'last_name': 'EATS', 'password': 'eat!'}}]}}
User email: eat!
Error querying Elasticsearch: NotFoundError(404, "{'_index': 'users', '_id': 'eat!', '_version': 1, 'result': 'not_found', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1}")


C:\Users\13307\AppData\Local\Temp\ipykernel_47840\3488772710.py:42: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  results = es.search(index="users", body=es_query)  # Assuming users are stored in 'users' index


In [37]:
# Get all users from the index
# Get all users from the index
try:
    results = es.search(index="users", body={
        "query": {
            "match_all": {}
        }
    }, size=100)  # Increase size if you have more users
    
    print("All Users:")
    for hit in results["hits"]["hits"]:
        user = hit["_source"]
        print(f"\nUsername: {user['username']}")
        print(f"Email: {user['email']}")
        print(f"Name: {user['first_name']} {user['last_name']}")
        print(f"Password: {user['password']}")
        print("-" * 50)
except Exception as e:
    print("Error querying Elasticsearch:", e)

All Users:

Username: user1
Email: user1@gmail.com
Name: John Doe
Password: password@1
--------------------------------------------------

Username: eat
Email: eat@gmail.com
Name: Eat EATS
Password: eat!
--------------------------------------------------


C:\Users\13307\AppData\Local\Temp\ipykernel_47840\4152322044.py:4: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  results = es.search(index="users", body={
